# Project: DrugGuardian Pro

## Setup

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

## Tensorflow with CUDA
Uncomment to force tensorflow to use CPU for training and testing

## Libraries

In [2]:
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [3]:
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve, auc, precision_recall_curve, f1_score, recall_score, precision_score
from sklearn.model_selection import train_test_split

from models import neural_binary_model, k_nearest_neighbors, linear_discriminant_analysis
from storage import save_data, load_data, save_label_encoder, load_label_encoder
from tools import prepare_dataset1, preprocess_labels

## Parameters

In [4]:
storage_path = "./storage/DS1/"

save_to_file = True
load_from_files = True

save_model_to_file = False
load_model_from_file = False

#Ignored if train and test sets are loaded from disk
n_sample = None #[set None to include the whole dataset] 

## Data Loading
Trys loading data from files on disk

In [5]:
if load_from_files:
    try:
        X_train, X_test, y_train, y_test = load_data(storage_path)
        encoder = load_label_encoder(storage_path)
    except FileNotFoundError:
        # disables loading from files (flag)
        load_from_files = False

## Data preparation
In case loading from files is disabled or unsuccessful

In [6]:
if not load_from_files:
    # Load your data and labels
    X, y = prepare_dataset1(sample_size=n_sample, keep_all_features=False, separate=True)

    # Convert labels to numpy array
    y, encoder = preprocess_labels(y)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Release memory by deleting the original data
    del X, y

    # Reshape the data
    X_train = X_train.reshape(X_train.shape[0], -1)
    X_test = X_test.reshape(X_test.shape[0], -1)

    if save_to_file:
        # Save the data and labels to files
        save_data(X_train, X_test, y_train, y_test, storage_path)
        save_label_encoder(encoder, storage_path)

        # Load the data and labels back from the files (for mmap_mode)
        X_train, X_test, y_train, y_test = load_data(storage_path)

## Model Fitting

### K-Nearest Neighbors


In [7]:
# Create a knn classifier instance
knn = k_nearest_neighbors()
# Train the model
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=21)

### Linear Discriminant Analysis

In [ ]:
# Create a lda classifier instance
lda = linear_discriminant_analysis()
# Train the model
lda.fit(X_train, encoder.inverse_transform((y_train > 0.5).astype(int).argmax(axis=1)))

LinearDiscriminantAnalysis()

### Neural Binary Model

In [9]:
# Create a neural network model
nbm = neural_binary_model(X_train.shape[1])
# Train the model
nbm.fit(X_train, y_train, epochs=20, batch_size=64, validation_split=0, shuffle=True)

Epoch 1/20
3754/3754 [==============================] - 9s 1ms/step - loss: 0.5116
Epoch 2/20
3754/3754 [==============================] - 6s 1ms/step - loss: 0.3260
Epoch 3/20
3754/3754 [==============================] - 5s 1ms/step - loss: 0.3049
Epoch 4/20
3754/3754 [==============================] - 6s 1ms/step - loss: 0.2940
Epoch 5/20
3754/3754 [==============================] - 6s 1ms/step - loss: 0.2873
Epoch 6/20
3754/3754 [==============================] - 5s 1ms/step - loss: 0.2803
Epoch 7/20
3754/3754 [==============================] - 6s 1ms/step - loss: 0.2743
Epoch 8/20
3754/3754 [==============================] - 5s 1ms/step - loss: 0.2698
Epoch 9/20
3754/3754 [==============================] - 5s 1ms/step - loss: 0.2637
Epoch 10/20
3754/3754 [==============================] - 5s 1ms/step - loss: 0.2594
Epoch 11/20
3754/3754 [==============================] - 5s 1ms/step - loss: 0.2533
Epoch 12/20
3754/3754 [==============================] - 6s 1ms/step - loss: 0.2482
E

## Evaluation

### K-Nearest Neighbors

In [10]:
# Evaluate the model on the test set
y_pred = knn.predict(X_test)

# Convert one-hot encoded labels back to original labels
y_test_labels = encoder.inverse_transform(y_test.argmax(axis=1))
y_pred_labels = encoder.inverse_transform(y_pred.argmax(axis=1))

# Evaluate the performance
accuracy = accuracy_score(y_test_labels, y_pred_labels)
print(f"Accuracy: {accuracy}")

confusion_matrix(y_test_labels, y_pred_labels)

Accuracy: 0.7647724813106674


array([[39650,   949],
       [13179,  6283]])

### Linear Discriminant Analysis

In [12]:
# Evaluate the model on the test set
y_pred = lda.predict(X_test)

# Convert one-hot encoded labels back to original labels
y_test_labels = encoder.inverse_transform(y_test.argmax(axis=1))

# Evaluate the performance
accuracy = accuracy_score(y_test_labels, y_pred)
print(f"Accuracy: {accuracy}")

confusion_matrix(y_test_labels, y_pred)

Accuracy: 0.5303275003746192


array([[22353, 18246],
       [ 9963,  9499]])

### Neural Binary Model

In [ ]:
# Evaluate the model on the test set
y_pred = nbm.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)

# Convert one-hot encoded labels back to original labels
y_test_labels = encoder.inverse_transform(y_test.argmax(axis=1))
y_pred_labels = encoder.inverse_transform(y_pred_binary.argmax(axis=1))

# Evaluate the performance
accuracy = accuracy_score(y_test_labels, y_pred_labels)
print(f"Accuracy: {accuracy}")

confusion_matrix(y_test_labels, y_pred_labels)

## Prediction example
Prediction will be performed for two drug pairs: Meperidine and Dexmedetomidine.

In [ ]:
# DB00454 (Meperidine) Feature vector
drug_1_sim = [   0.00004,   0.00012,   0.00011,   0.00012,   0.84896,   0.00008,   0.00005,   0.00009,   0.00009,   0.00011,   0.00010,   0.00011,   0.00009,   0.00007,   0.00008,   0.00008,   0.00012,   0.00007,   0.00010,   0.00009,   0.00010,   0.00006,   0.00011,   0.00007,   0.00006,   0.00007,   0.00006,   0.00012,   0.00009,   0.00006,   0.00006,   0.00280,   0.00006,   0.00008,   0.00007,   0.00008,   0.00010,   0.00005,   0.00010,   0.00010,   0.00008,   0.00009,   0.00007,   0.00004,   0.00010,   0.00007,   0.00007,   0.00007,   0.00006,   0.00010,   0.00007,   0.00012,   0.00006,   0.00005,   0.00008,   0.00012,   0.00013,   0.00005,   0.00008,   0.00011,   0.00012,   0.00010,   0.00006,   0.00009,   0.00008,   0.00011,   0.00005,   0.00009,   0.00009,   0.00009,   0.00006,   0.00005,   0.00007,   0.00007,   0.00005,   0.00011,   0.00011,   0.00012,   0.00008,   0.00008,   0.00012,   0.00008,   0.00009,   0.00007,   0.00010,   0.00010,   0.00006,   0.00009,   0.00010,   0.00005,   0.00008,   0.00012,   0.00006,   0.00005,   0.00011,   0.00039,   0.00006,   0.00008,   0.00009,   0.00007,   0.00012,   0.00007,   0.00012,   0.00010,   0.00012,   0.00009,   0.00008,   0.00006,   0.00006,   0.00010,   0.00011,   0.00007,   0.00010,   0.00013,   0.00006,   0.00007,   0.00007,   0.00007,   0.00011,   0.00006,   0.00008,   0.00005,   0.00006,   0.00009,   0.00005,   0.00012,   0.00011,   0.04033,   0.00011,   0.00011,   0.00010,   0.00004,   0.00006,   0.00011,   0.00006,   0.00007,   0.00010,   0.00008,   0.00009,   0.00007,   0.00005,   0.00734,   0.00099,   0.00007,   0.00008,   0.00007,   0.00007,   0.00010,   0.00006,   0.00008,   0.00007,   0.00006,   0.00010,   0.00005,   0.00010,   0.00010,   0.00011,   0.00008,   0.00004,   0.00039,   0.00006,   0.00009,   0.00007,   0.00012,   0.00009,   0.00005,   0.00012,   0.00012,   0.00012,   0.00007,   0.00010,   0.00008,   0.00008,   0.00011,   0.00005,   0.00008,   0.00011,   0.00010,   0.00099,   0.00009,   0.00009,   0.00011,   0.00011,   0.00009,   0.00009,   0.00009,   0.00012,   0.00012,   0.00009,   0.00009,   0.00007,   0.01886,   0.00006,   0.00006,   0.00007,   0.04033,   0.00011,   0.00010,   0.00008,   0.00008,   0.00010,   0.00007,   0.00009,   0.00012,   0.00007,   0.00012,   0.00006,   0.00010,   0.00005,   0.00005,   0.00008,   0.00008,   0.00013,   0.00011,   0.00014,   0.00009,   0.00010,   0.00008,   0.00010,   0.00007,   0.00009,   0.00009,   0.00008,   0.00008,   0.00008,   0.00011,   0.00007,   0.00010,   0.00008,   0.00007,   0.00010,   0.00007,   0.00009,   0.00010,   0.00008,   0.00010,   0.00009,   0.00005,   0.00009,   0.00008,   0.00010,   0.00010,   0.00009,   0.00010,   0.00011,   0.00006,   0.00006,   0.00007,   0.00008,   0.00011,   0.00007,   0.00006,   0.00006,   0.00008,   0.00012,   0.00007,   0.00008,   0.00008,   0.00005,   0.00005,   0.00007,   0.00012,   0.00008,   0.00006,   0.00005,   0.00005,   0.00008,   0.00007,   0.00008,   0.00012,   0.00007,   0.00011,   0.00006,   0.00014,   0.00008,   0.00008,   0.00005,   0.00012,   0.00006,   0.00007,   0.00011,   0.00007,   0.00009,   0.00007,   0.00010,   0.00007,   0.00011,   0.00012,   0.00008,   0.01886,   0.00010,   0.00010,   0.00007,   0.00006,   0.00008,   0.00012,   0.00007,   0.00010,   0.00006,   0.00009,   0.00006,   0.00010,   0.00012,   0.00007,   0.00007,   0.00006,   0.00013,   0.00009,   0.00008,   0.00007,   0.00007,   0.00005,   0.00005,   0.00006,   0.00007,   0.00010,   0.00008,   0.00009,   0.00009,   0.00006,   0.00008,   0.00005,   0.00008,   0.00007,   0.00007,   0.00011,   0.00007,   0.00010,   0.00013,   0.00012,   0.00012,   0.00008,   0.00008,   0.00011,   0.00011,   0.00010,   0.00010,   0.00005,   0.00013,   0.00005,   0.00008,   0.14047,   0.00010,   0.00010,   0.00013,   0.00008,   0.00008,   0.00008,   0.00009,   0.00011,   0.00009,   0.00007,   0.00010,   0.00011,   0.00006,   0.00008,   0.00008,   0.00011,   0.00009,   0.00007,   0.00004,   0.00009,   0.00020,   0.00007,   0.00006,   0.00010,   0.00007,   0.00006,   0.00006,   0.00008,   0.00005,   0.00010,   0.00010,   0.00008,   0.00006,   0.00011,   0.00006,   0.00012,   0.00009,   0.00009,   0.00008,   0.00010,   0.00009,   0.00007,   0.00011,   0.00009,   0.00006,   0.00011,   0.00012,   0.00009,   0.00008,   0.00006,   0.00007,   0.00012,   0.00005,   0.00009,   0.00008,   0.00011,   0.00007,   0.00008,   0.00007,   0.00009,   0.00008,   0.00006,   0.00009,   0.00010,   0.00012,   0.00010,   0.00010,   0.00008,   0.00007,   0.00008,   0.00020,   0.00008,   0.00009,   0.00009,   0.00011,   0.00009,   0.00012,   0.00007,   0.00012,   0.00005,   0.00006,   0.00009,   0.00008,   0.00006,   0.00005,   0.00013,   0.00009,   0.00011,   0.00006,   0.00008,   0.00008,   0.00011,   0.00005,   0.00009,   0.00007,   0.00012,   0.00011,   0.00010,   0.00007,   0.00007,   0.00008,   0.00009,   0.00007,   0.00005,   0.00008,   0.00004,   0.00010,   0.00005,   0.00013,   0.00012,   0.00008,   0.00007,   0.00012,   0.00010,   0.00006,   0.00009,   0.00010,   0.00007,   0.00006,   0.00011,   0.00012,   0.00007,   0.00010,   0.09169,   0.00005,   0.00008,   0.00008,   0.00007,   0.00008,   0.00009,   0.00010,   0.00005,   0.00009,   0.00012,   0.00011,   0.00009,   0.00011,   0.00012,   0.00009,   0.00008,   0.09169,   0.00005,   0.00006,   0.00010,   0.00012,   0.00006,   0.00011,   0.00009,   0.00004,   0.00005,   0.00006,   0.00012,   0.00012,   0.00009,   0.00006,   0.00006,   0.00006,   0.00007,   0.00006,   0.00006,   0.00011,   0.00007,   0.00011,   0.00009,   0.00007,   0.00010,   0.00009,   0.00011,   0.00005,   0.00011,   0.00006,   0.00008,   0.00009,   0.00006,   0.00009,   0.00007,   0.00007,   0.00006,   0.00007,   0.00008,   0.00009,   0.00006,   0.14047,   0.00011,   0.00007,   0.00735,   0.00008,   0.00011,   0.00007,   0.00011,   0.00005,   0.00007,   0.00008,   0.00007,   0.00009,   0.00007,   0.00009,   0.00009,   0.00006,   0.00007,   0.00009,   0.00008,   0.00005,   0.00280,   0.00009,   0.00012]

# DB00633 (Dexmedetomidine) Feature vector
drug_2_sim = [   0.00002,   0.00006,   0.00006,   0.00006,   0.00007,   0.00016,   0.00003,   0.00011,   0.00005,   0.00006,   0.00005,   0.00008,   0.00011,   0.00044,   0.00004,   0.00004,   0.00008,   0.00004,   0.00010,   0.00018,   0.00010,   0.00003,   0.00009,   0.04014,   0.00003,   0.00004,   0.00003,   0.00008,   0.00012,   0.00003,   0.00003,   0.00007,   0.00003,   0.00016,   0.85059,   0.00013,   0.00010,   0.00029,   0.00010,   0.00006,   0.00004,   0.00011,   0.00004,   0.00002,   0.00005,   0.00016,   0.00003,   0.00004,   0.00003,   0.00005,   0.14089,   0.00008,   0.00003,   0.00002,   0.00005,   0.00008,   0.00007,   0.00003,   0.00004,   0.00009,   0.00007,   0.00010,   0.00003,   0.00005,   0.00013,   0.00006,   0.00003,   0.00011,   0.00012,   0.00005,   0.00003,   0.00003,   0.00017,   0.04062,   0.00003,   0.00006,   0.00009,   0.00006,   0.00014,   0.00014,   0.00008,   0.00016,   0.00012,   0.00004,   0.00005,   0.00009,   0.00003,   0.00005,   0.00010,   0.00003,   0.00014,   0.00008,   0.00003,   0.00002,   0.00006,   0.00007,   0.00003,   0.00013,   0.00012,   0.01859,   0.00006,   0.00017,   0.00006,   0.00006,   0.00008,   0.00012,   0.00015,   0.00003,   0.00003,   0.00005,   0.00006,   0.00004,   0.00009,   0.00007,   0.00003,   0.00004,   0.00017,   0.00016,   0.00009,   0.00003,   0.00013,   0.00003,   0.00003,   0.00017,   0.00003,   0.00008,   0.00009,   0.00007,   0.00009,   0.00006,   0.00005,   0.00002,   0.00003,   0.00009,   0.00003,   0.00025,   0.00009,   0.00004,   0.00005,   0.00004,   0.00003,   0.00007,   0.00007,   0.00004,   0.00013,   0.00004,   0.00004,   0.00006,   0.00003,   0.00013,   0.00017,   0.00003,   0.00005,   0.00003,   0.00005,   0.00010,   0.00006,   0.00004,   0.00002,   0.00007,   0.00003,   0.00012,   0.00004,   0.00008,   0.00005,   0.00003,   0.00006,   0.00008,   0.00007,   0.00003,   0.00010,   0.00004,   0.00013,   0.00009,   0.00003,   0.00004,   0.00006,   0.00010,   0.00007,   0.00005,   0.00011,   0.00006,   0.00008,   0.00011,   0.00012,   0.00012,   0.00008,   0.00008,   0.00005,   0.00005,   0.00004,   0.00007,   0.00003,   0.00003,   0.00004,   0.00007,   0.00006,   0.00009,   0.00013,   0.00017,   0.00005,   0.00004,   0.00011,   0.00008,   0.00017,   0.00007,   0.00003,   0.00010,   0.00003,   0.00003,   0.00014,   0.00014,   0.00007,   0.00006,   0.00007,   0.00005,   0.00005,   0.00004,   0.00010,   0.00004,   0.00012,   0.00018,   0.00004,   0.00014,   0.00013,   0.00006,   0.00105,   0.00009,   0.00005,   0.00004,   0.00005,   0.00004,   0.00005,   0.00011,   0.00004,   0.00011,   0.00012,   0.00003,   0.00012,   0.00013,   0.00010,   0.00005,   0.00011,   0.00006,   0.00009,   0.00003,   0.00003,   0.00004,   0.00014,   0.00006,   0.00004,   0.00003,   0.00003,   0.00004,   0.00006,   0.00018,   0.00013,   0.00004,   0.00029,   0.00003,   0.00004,   0.00008,   0.00004,   0.00003,   0.00003,   0.00003,   0.00004,   0.00003,   0.00014,   0.00008,   0.00004,   0.00008,   0.00003,   0.00007,   0.00004,   0.00014,   0.00029,   0.00007,   0.00003,   0.00004,   0.00006,   0.00004,   0.00012,   0.00004,   0.00005,   0.00004,   0.00009,   0.00007,   0.00014,   0.00007,   0.00005,   0.00010,   0.00017,   0.00003,   0.00004,   0.00008,   0.00016,   0.00010,   0.00003,   0.00005,   0.00003,   0.00005,   0.00008,   0.00004,   0.00004,   0.00003,   0.00007,   0.00011,   0.00004,   0.00016,   0.00745,   0.00003,   0.00003,   0.00003,   0.00004,   0.00005,   0.00004,   0.00012,   0.00005,   0.00003,   0.00015,   0.00003,   0.00014,   0.00003,   0.01904,   0.00009,   0.00004,   0.00005,   0.00007,   0.00007,   0.00008,   0.00014,   0.00004,   0.00009,   0.00009,   0.00010,   0.00010,   0.00003,   0.00007,   0.00003,   0.00004,   0.00007,   0.00005,   0.00005,   0.00007,   0.00004,   0.00004,   0.00013,   0.00011,   0.00009,   0.00011,   0.00003,   0.00006,   0.00009,   0.00003,   0.00013,   0.00014,   0.00009,   0.00011,   0.00016,   0.00002,   0.00011,   0.00007,   0.00004,   0.00003,   0.00010,   0.00004,   0.00050,   0.00003,   0.00004,   0.00003,   0.00005,   0.00009,   0.00004,   0.00003,   0.00006,   0.00003,   0.00008,   0.00005,   0.00005,   0.00016,   0.00010,   0.00005,   0.00017,   0.00009,   0.00005,   0.00273,   0.00009,   0.00008,   0.00005,   0.00015,   0.00003,   0.00019,   0.00006,   0.00003,   0.00005,   0.00004,   0.00006,   0.00004,   0.00004,   0.00004,   0.00005,   0.00004,   0.00003,   0.00013,   0.00010,   0.00006,   0.00005,   0.00010,   0.00004,   0.00017,   0.00014,   0.00007,   0.00015,   0.00012,   0.00005,   0.00006,   0.00005,   0.00006,   0.14115,   0.00006,   0.00003,   0.00003,   0.00011,   0.00015,   0.00003,   0.00003,   0.00007,   0.00005,   0.00006,   0.00003,   0.00015,   0.00004,   0.00009,   0.00003,   0.00005,   0.00004,   0.00007,   0.00006,   0.00006,   0.00016,   0.09180,   0.00004,   0.00012,   0.00004,   0.00003,   0.00004,   0.00002,   0.00009,   0.00003,   0.00007,   0.00008,   0.00013,   0.00017,   0.00006,   0.00005,   0.00103,   0.00005,   0.00005,   0.00004,   0.00003,   0.00008,   0.00008,   0.00004,   0.00010,   0.00007,   0.00003,   0.00013,   0.00004,   0.00716,   0.00004,   0.00012,   0.00010,   0.00003,   0.00011,   0.00006,   0.00006,   0.00005,   0.00006,   0.00007,   0.00005,   0.00013,   0.00007,   0.00003,   0.00003,   0.00010,   0.00008,   0.00003,   0.00008,   0.00012,   0.00002,   0.00003,   0.00003,   0.00008,   0.00006,   0.00005,   0.00003,   0.00003,   0.00003,   0.00004,   0.00003,   0.00003,   0.00009,   0.00004,   0.00009,   0.00011,   0.00016,   0.00010,   0.00012,   0.00008,   0.00029,   0.00008,   0.00003,   0.00004,   0.00005,   0.00003,   0.00005,   0.00004,   0.00288,   0.00030,   0.00017,   0.00005,   0.00005,   0.00034,   0.00007,   0.00006,   0.00004,   0.00007,   0.00004,   0.00006,   0.00004,   0.00006,   0.00003,   0.00004,   0.00014,   0.00017,   0.00005,   0.09228,   0.00012,   0.00011,   0.00003,   0.00018,   0.00011,   0.00016,   0.00003,   0.00007,   0.00011,   0.00008
]

drugs_comb = np.concatenate((drug_1_sim, drug_2_sim))
drugs_comb = drugs_comb.reshape(1, -1)

pred_binary = (nbm.predict(drugs_comb) > 0.5).astype(int)

# Convert one-hot encoded labels back to original labels
pred_label = encoder.inverse_transform(pred_binary.argmax(axis=1))

pred_label

The results obtained above indicate a prediction of "False," suggesting that there are no interactions between the specified drug pairs.